# import packages

In [1]:
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from tools import *
from multiprocessing import Pool
import os
import itertools

/opt/conda/envs/autocell-web-ui/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data and QC processing

In [2]:
adata_raw = read_h5ad('path_to_data/adata.h5ad')
mito_genes = adata_raw.var['gene_ids'].str.startswith('MT-')
adata_raw.obs['n_genes'] = sum(adata_raw.X > 0, axis=1)
adata_raw.var['n_cells'] = sum(adata_raw.X > 0, axis=0).T
if type(adata_raw.X) == np.ndarray:
    adata_raw.obs['percent_mito'] = np.sum(
        adata_raw[:, mito_genes].X, axis=1) / np.sum(adata_raw.X, axis=1)
    adata_raw.obs['n_counts'] = adata_raw.X.sum(axis=1)
else:
    adata_raw.obs['percent_mito'] = np.sum(adata_raw[:, mito_genes].X, axis=1).A1 / np.sum(adata_raw.X, axis=1).A1
    adata_raw.obs['n_counts'] = adata_raw.X.sum(axis=1).A1

In [3]:
adata = filter(adata_raw, min_genes=200, min_counts=1000, min_cells=2)

started with  9531  total cells and  32738  total genes
removed 0 cells that did not express at least 200  genes
removed 0 cells that did not have at least 1000 counts
removed 13016 genes that were not expressed in at least 2 cells
finished with 9531  total cells and 19722 total genes


# Analytic Pearson residual normalization

In [4]:
adata.var['qc_gene_idx'] = list(range(len(adata.var))
z = analytic_pearson_residuals(scipy.sparse.csr_matrix(adata.X), 100)
var = np.squeeze(np.asarray(np.var(z,axis=0)))
adata.var['pr_var'] = var 
means_exp= np.mean(adata.X,axis=0)
adata.var['mean_expression'] = np.squeeze(np.asarray(means_exp))
adata_apr = adata.copy()
adata.X = z

# Selection of analytic Pearson residual variated genes

In [5]:
aprg_mask = (adata.var.pr_var > 1.3) & (adata.var.mean_expression > 0.0008)

# Calculation of batch expression descriptors

In [ ]:
adata.obs['qc_cell_idx'] = list(range(len(adata.obs)))
batch_label = 'Batch'
path  = './{}/batch_effect_feature'.format(batch_label)
if not os.path.exists(path):
    os.makedirs(path)
batch = np.unique(adata.obs[batch_label])

pair = []
for pair_ in itertools.combinations(batch,2):
    pair.append([pair_[0],pair_[1]])

def get_batch_expression_descriptors(batch):
    adata_X = adata.X
    batch_id_a = batch
    if 'qc_cell_idx' not in adata.obs.columns:
        print('cell index is not define!')
    else:
        df_a = get_nb_parameter_df(adata_X[adata.obs.loc[adata.obs[batch_label] == batch_id_a].qc_cell_idx,:],aprg_mask)
        df_a.to_excel(os.path.join(path,'{}.xlsx'.format(batch_id_a)))
        print('excel file of batch {} has saved!'.format(batch_id_a))
pool = Pool(36)                        
pool.map(get_batch_expression_descriptors,batch) 
pool.close()
pool.join()

# Caculation of cosine distance between each two batches of each APR-variated gene

In [ ]:
path_  = './{}/cos_similarity'.format(batch)
if not os.path.exists(path_):
    os.makedirs(path_)
    
def get_batch_cos_sim(pair):
    sample_id_a,sample_id_b = pair
    df_a = pd.read_excel(os.path.join(path,sample_id_a+'.xlsx'))
    df_b = pd.read_excel(os.path.join(path,sample_id_b+'.xlsx'))
    df_ab = gene_batch_effect_profiling(df_a,df_b,sample_id_a,sample_id_b)
    df_ab.to_excel(os.path.join(path_,sample_id_a+'_vs_'+ sample_id_b +'.xlsx'))
    print('calculation of cos between {} and {} is done and saved!'.format(sample_id_a,sample_id_b))

pool = Pool(36)                      
pool.map(get_batch_cos_sim,pair) 
pool.close()
pool.join()

cs_dflist = []
corr_dflist = []
for i in range(len(pair)):
    sample_a, sample_b = pair[i]
    filename = sample_a + '_vs_' + sample_b + '.xlsx'
    df_ = pd.read_excel(os.path.join(path_,filename))
    cs_dflist.append(df_.iloc[:,1])
    corr_dflist.append(df_.iloc[:,2])
df_cs = pd.concat(cs_dflist,axis=1)
df_r = pd.concat(corr_dflist,axis=1)

# Selection of highly variable genes

In [ ]:
df_cs.index = adata[:,aprg_mask].var.gene_ids.values
df_rbg = df_cs.loc[df_cs.index.str.startswith('RPS') | df_cs.index.str.startswith('RPL')]
relative_cs = (df_cs  - df_rbg.mean()).fillna(0).abs()
bio_intensity = relative_cs.sum(axis=1)
hvg_df = pd.DataFrame(bio_intensity)
hvg_df['qc_gene_idx'] = adata[:,aprg_mask].var.qc_gene_idx.values
adata_hvg = adata_apr[:,hvg_df[hvg_df.bio_intensity > 2.2].qc_gene_idx]

# Dimension reduction and clustering

In [ ]:
adata_hvg = sc.pp.scale(adata_hvg,zero_center=True, max_value=3,copy=True)
adata_latent = dim_reduction_ae(adata_hvg)
adata.obs['ae'] = adata_latent.obs.autoencoder.values

# UMAP visulization

In [ ]:
sc.pl.umap(adata_latent,color=['ae'],legend_loc='on data',frameon=False,add_outline=True,legend_fontsize=8)
sc.pl.umap(adata_latent,color=['Cell_type'],frameon=False,legend_fontoutline=1,add_outline=True,legend_fontsize=6)